<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/cifar100/CIFAR100_do_label_shift_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!wget https://zenodo.org/record/3458743/files/am_cifar100_test_labels.txt.gz?download=1 -O am_cifar100_test_labels.txt.gz
!wget https://zenodo.org/record/3458743/files/am_cifar100_valid_labels.txt.gz?download=1 -O am_cifar100_valid_labels.txt.gz
!wget https://zenodo.org/record/3458743/files/cifar100_validpreacts_seed0.txt.gz?download=1 -O cifar100_validpreacts_seed0.txt.gz
!wget https://zenodo.org/record/3458743/files/cifar100_testpreacts_seed0.txt.gz?download=1 -O cifar100_testpreacts_seed0.txt.gz
!wget https://zenodo.org/record/3458743/files/cifar100_validpreacts_seed10.txt.gz?download=1 -O cifar100_validpreacts_seed10.txt.gz
!wget https://zenodo.org/record/3458743/files/cifar100_testpreacts_seed10.txt.gz?download=1 -O cifar100_testpreacts_seed10.txt.gz
!wget https://zenodo.org/record/3458743/files/cifar100_validpreacts_seed20.txt.gz?download=1 -O cifar100_validpreacts_seed20.txt.gz
!wget https://zenodo.org/record/3458743/files/cifar100_testpreacts_seed20.txt.gz?download=1 -O cifar100_testpreacts_seed20.txt.gz
!wget https://zenodo.org/record/3458743/files/cifar100_validpreacts_seed30.txt.gz?download=1 -O cifar100_validpreacts_seed30.txt.gz
!wget https://zenodo.org/record/3458743/files/cifar100_testpreacts_seed30.txt.gz?download=1 -O cifar100_testpreacts_seed30.txt.gz
!wget https://zenodo.org/record/3458743/files/cifar100_validpreacts_seed40.txt.gz?download=1 -O cifar100_validpreacts_seed40.txt.gz
!wget https://zenodo.org/record/3458743/files/cifar100_testpreacts_seed40.txt.gz?download=1 -O cifar100_testpreacts_seed40.txt.gz

--2019-09-24 05:46:23--  https://zenodo.org/record/3458743/files/am_cifar100_test_labels.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46062 (45K) [application/octet-stream]
Saving to: ‘am_cifar100_test_labels.txt.gz’

am_cifar100_test_la 100%[===================>]  44.98K   145KB/s    in 0.3s    

2019-09-24 05:46:25 (145 KB/s) - ‘am_cifar100_test_labels.txt.gz’ saved [46062/46062]

--2019-09-24 05:46:26--  https://zenodo.org/record/3458743/files/am_cifar100_valid_labels.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46020 (45K) [application/octet-stream]
Saving to: ‘am_cifar100_valid_labels.txt.gz’

am_cifar100_valid_l 100%[===================>]  44.94K   145KB/s    in 0.3s    

2019-09-24 05

In [0]:
![[ -e abstention ]] || git clone https://github.com/blindauth/abstention
%cd /content/abstention
!git pull
!pip uninstall abstention
!pip install .
%cd ..

/content/abstention
Already up to date.
Uninstalling abstention-0.1.3.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/abstention-0.1.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/abstention/*
Proceed (y/n)? y
  Successfully uninstalled abstention-0.1.3.0
Processing /content/abstention
  Created wheel for abstention: filename=abstention-0.1.3.0-cp36-none-any.whl size=46751 sha256=e2ea2b8b60090711617df6f44d5d28c2fb85aeaccaa3489bd0f6157bade7ba37
  Stored in directory: /tmp/pip-ephem-wheel-cache-szr5ild1/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
/content


In [0]:
![[ -e label_shift_experiments ]] || git clone https://github.com/blindauth/labelshiftexperiments
%cd /content/labelshiftexperiments
!git pull
!pip uninstall labelshiftexperiments
!pip install .
%cd ..

fatal: destination path 'labelshiftexperiments' already exists and is not an empty directory.
/content/labelshiftexperiments
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/blindauth/labelshiftexperiments
   781f678..59b97bb  master     -> origin/master
Updating 781f678..59b97bb
Fast-forward
 labelshiftexperiments/cifarandmnist.py | 9 +++++++++
 1 file changed, 9 insertions(+)
Uninstalling labelshiftexperiments-0.1.0.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/labelshiftexperiments-0.1.0.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/labelshiftexperiments/*
Proceed (y/n)? y
  Successfully uninstalled labelshiftexperiments-0.1.0.0
Processing /content/labelshiftexperiments
  Created wheel for labelshiftexperiments: filename=labelshiftexperiments-0.1.0.0-cp36-no

In [0]:
!rm *.txt
!gunzip *.gz

In [10]:
!ls

abstention			 cifar100_validpreacts_seed0.txt
am_cifar100_test_labels.txt	 cifar100_validpreacts_seed10.txt
am_cifar100_valid_labels.txt	 cifar100_validpreacts_seed20.txt
cifar100_testpreacts_seed0.txt	 cifar100_validpreacts_seed30.txt
cifar100_testpreacts_seed10.txt  cifar100_validpreacts_seed40.txt
cifar100_testpreacts_seed20.txt  label_shift_adaptation_results.json
cifar100_testpreacts_seed30.txt  labelshiftexperiments
cifar100_testpreacts_seed40.txt  sample_data


In [11]:
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
reload(abstention.figure_making_utils)
from abstention.calibration import (
    TempScaling, VectorScaling, NoBiasVectorScaling, softmax)
from abstention.label_shift import (EMImbalanceAdapter,
     BBSEImbalanceAdapter, ShiftWeightFromImbalanceAdapter)
import glob
import gzip
import numpy as np
from collections import defaultdict, OrderedDict

import labelshiftexperiments
reload(labelshiftexperiments)
reload(labelshiftexperiments.cifarandmnist)
from labelshiftexperiments import cifarandmnist
        
test_labels = cifarandmnist.read_preds(open("am_cifar100_test_labels.txt"))
valid_labels = cifarandmnist.read_preds(open("am_cifar100_valid_labels.txt"))

imbalanceadaptername_to_imbalanceadapter = {
    'em': EMImbalanceAdapter(),
    'bbse-hard': BBSEImbalanceAdapter(soft=False),
    'bbse-soft': BBSEImbalanceAdapter(soft=True)}

calibname_to_calibfactory = OrderedDict([
    ('None', abstention.calibration.Softmax()),
    ('TS', TempScaling(verbose=False)),
    ('NBVS', NoBiasVectorScaling(verbose=False)),
    ('BCTS', TempScaling(verbose=False,
                         bias_positions='all')),
    ('VS', VectorScaling(verbose=False))
])

adaptncalib_pairs = [
    ('bbse-hard', 'None'),
    ('bbse-soft', 'None'),
    ('bbse-soft', 'TS'),
    ('bbse-soft', 'NBVS'),
    ('bbse-soft', 'BCTS'),
    ('bbse-soft', 'VS'),
    ('bbse-soft', 'best-ece'),
    ('bbse-soft', 'best-jsdiv'),
    ('bbse-soft', 'best-nll'),
    ('em', 'None'),
    ('em', 'TS'),
    ('em', 'NBVS'),
    ('em', 'BCTS'),
    ('em', 'VS'),
    ('em', 'best-ece'),
    ('em', 'best-jsdiv'),
    ('em', 'best-nll')
]

num_trials = 10
seeds = [0, 10, 20, 30, 40]
dirichlet_alphas_and_samplesize = [(1.0,7000), (1.0,8500), (1.0,10000),
                                   (0.1,7000), (0.1,8500), (0.1,10000)]
tweakone_alphas_and_samplesize = []

print("Dirichlet shift")

(dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,      
 dirichlet_alpha_to_samplesize_to_baselineacc,                        
 metric_to_samplesize_to_calibname_to_unshiftedvals) =\
  cifarandmnist.run_experiments(
    num_trials=num_trials,
    seeds=seeds,
    alphas_and_samplesize = dirichlet_alphas_and_samplesize,
    shifttype='dirichlet',
    calibname_to_calibfactory=calibname_to_calibfactory,
    imbalanceadaptername_to_imbalanceadapter=
      imbalanceadaptername_to_imbalanceadapter,
    adaptncalib_pairs=adaptncalib_pairs,
    validglobprefix="cifar100_validpreacts_seed",
    testglobprefix="cifar100_testpreacts_seed",
    valid_labels=valid_labels,
    test_labels=test_labels)

print("Tweak one shift")

(tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,      
 tweakone_alpha_to_samplesize_to_baselineacc,                        
 _) = cifarandmnist.run_experiments(
    num_trials=num_trials,
    seeds=seeds,
    alphas_and_samplesize = tweakone_alphas_and_samplesize,
    shifttype='tweakone',
    calibname_to_calibfactory=calibname_to_calibfactory,
    imbalanceadaptername_to_imbalanceadapter=
      imbalanceadaptername_to_imbalanceadapter,
    adaptncalib_pairs=adaptncalib_pairs,
    validglobprefix="cifar100_validpreacts_seed",
    testglobprefix="cifar100_testpreacts_seed",
    valid_labels=valid_labels,
    test_labels=test_labels)



Dirichlet shift
Seed 0
Seed 10
Seed 20
Seed 30
Seed 40
Calibration stats
Metric ece
None 20.154212732188647
TS 3.0635847541238825
NBVS 2.346634287489793
BCTS 2.928126445110297
VS 2.516644363133144
Metric nll
None 1.7387520684222495
TS 1.2898810512668677
NBVS 1.2435182528379378
BCTS 1.2365366304566534
VS 1.2353663690019017
Metric jsdiv
None 0.0827000895514515
TS 0.07195624289048086
NBVS 0.031879676409035794
BCTS 0.030745741983396743
VS 0.03065280716218026
On alpha 1.0 sample size 7000
Metric delta_acc
bbse-hard:None 0.14884 +/- 0.0030121083979912614
bbse-soft:None 0.15290285714285712 +/- 0.0034130636650729056
bbse-soft:TS 0.20811999999999997 +/- 0.0041307500985305065
bbse-soft:NBVS 0.2119685714285714 +/- 0.0041078571618676524
bbse-soft:BCTS 0.21380857142857146 +/- 0.004356462901587856
bbse-soft:VS 0.21321428571428572 +/- 0.00409721566146976
bbse-soft:best-ece 0.2126457142857143 +/- 0.004144562311192051
bbse-soft:best-jsdiv 0.21226285714285717 +/- 0.004214657511889186
bbse-soft:best-nll 

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


Seed 10
Seed 20
Seed 30
Seed 40
On alpha 0.1 sample size 8500
Metric delta_acc
bbse-hard:None 0.1719129411764706 +/- 0.0076644447739243225
bbse-soft:None 0.17840235294117648 +/- 0.00722331362711512
bbse-soft:TS 0.2519341176470588 +/- 0.01231300336820729
bbse-soft:NBVS 0.26383529411764706 +/- 0.014099699275753727
bbse-soft:BCTS 0.2647270588235294 +/- 0.01461841307651567
bbse-soft:VS 0.26804941176470587 +/- 0.015253725992543706
bbse-soft:best-ece 0.26383529411764706 +/- 0.014099699275753727
bbse-soft:best-jsdiv 0.26804941176470587 +/- 0.015253725992543706
bbse-soft:best-nll 0.26804941176470587 +/- 0.015253725992543706
em:None 0.14651764705882353 +/- 0.007282736735635015
em:TS 0.2628541176470588 +/- 0.013169549635820437
em:NBVS 0.2649694117647059 +/- 0.014612474367689686
em:BCTS 0.2683882352941176 +/- 0.015265181325244464
em:VS 0.27288 +/- 0.01567209840265914
em:best-ece 0.2649694117647059 +/- 0.014612474367689686
em:best-jsdiv 0.27288 +/- 0.01567209840265914
em:best-nll 0.27288 +/- 0.015

In [12]:
import json
import os
file_out = "label_shift_adaptation_results.json"
dict_to_write = {
    "dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "dirichlet_alpha_to_samplesize_to_baselineacc":
     dirichlet_alpha_to_samplesize_to_baselineacc,
    "metric_to_samplesize_to_calibname_to_unshiftedvals":
     metric_to_samplesize_to_calibname_to_unshiftedvals,
    "tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "tweakone_alpha_to_samplesize_to_baselineacc":
     tweakone_alpha_to_samplesize_to_baselineacc
}
open(file_out, 'w').write(
    json.dumps(dict_to_write,
               sort_keys=True, indent=4, separators=(',', ': ')))
os.system("gzip -f "+file_out)

0

In [0]:
import gzip
import json
loaded_dicts = json.loads(gzip.open("label_shift_adaptation_results.json.gz").read())
dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
dirichlet_alpha_to_samplesize_to_baselineacc =\
    loaded_dicts['dirichlet_alpha_to_samplesize_to_baselineacc']
tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
tweakone_alpha_to_samplesize_to_baselineacc =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_baselineacc']
metric_to_samplesize_to_calibname_to_unshiftedvals =\
    loaded_dicts['metric_to_samplesize_to_calibname_to_unshiftedvals']

In [14]:
from importlib import reload
import labelshiftexperiments
reload(labelshiftexperiments)
import labelshiftexperiments.maketable
reload (labelshiftexperiments.maketable)
from labelshiftexperiments.maketable import render_calibration_table

metricname_to_nicename = {'nll': 'nll', 'jsdiv': 'jsdiv', 'ece': 'ECE'}
calibname_to_nicename = {'None': "None", "TS": "TS",
                         "VS":"VS", "NBVS": "NBVS", "BCTS": "BCTS"}
  
from scipy.stats import norm
N = len(seeds)*num_trials
#Using the normal approximation at N=100;
# variance from https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test
#Note that T = ((N+1)*N/2 - W)/2            
ustat_threshold = ((N*(N+1))/2 - norm.ppf(0.99)*np.sqrt(N*(N+1)*(2*N+1)/6.0))/2.0
  
print(render_calibration_table(
        metric_to_samplesize_to_calibname_to_unshiftedvals=
          metric_to_samplesize_to_calibname_to_unshiftedvals,
        #threshold of 8 comes from table https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
        #for one-tailed alpha=0.025 and n=10            
        ustat_threshold=ustat_threshold,
        metrics_in_table=['nll', 'ece'],
        samplesizes_in_table=['7000', '8500', '10000'],
        calibnames_in_table=['None', 'TS', 'NBVS', 'BCTS', 'VS'],
        metricname_to_nicename=metricname_to_nicename,
        calibname_to_nicename=calibname_to_nicename,
        caption="CIFAR100 Calibration metric differences", label="cifar10calibrationcomparison",
        applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c c c | c c c }
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{nll} & \multicolumn{3}{| c}{ECE}\\
    \cline{2-7}
    & $n$=7000 & $n$=8500 & $n$=10000 & $n$=7000 & $n$=8500 & $n$=10000\\
    \hline
    None & 1.739; 4.0 & 1.739; 4.0 & 1.739; 4.0 & 20.154; 4.0 & 20.154; 4.0 & 20.154; 4.0\\
    TS & 1.29; 3.0 & 1.29; 3.0 & 1.29; 3.0 & 3.064; 2.78 & 3.071; 2.74 & 3.056; 2.8\\
    NBVS & 1.244; 2.0 & 1.242; 2.0 & 1.241; 2.0 & \textbf{2.347; 0.12} & \textbf{2.34; 0.12} & \textbf{2.364; 0.2}\\
    BCTS & 1.237; 0.9 & 1.235; 1.0 & 1.234; 1.0 & 2.928; 2.2 & 2.941; 2.26 & 2.937; 2.2\\
    VS & \textbf{1.235; 0.1} & \textbf{1.233; 0.0} & \textbf{1.231; 0.0} & 2.517; 0.9 & 2.513; 0.88 & 2.486; 0.8\\
  \end{tabular}}
  \caption{CIFAR100 Calibration metric differences}
  \label{tab:cifar10calibrationcomparison}
\end{table*}



In [19]:
from labelshiftexperiments.maketable import render_adaptation_table
    
methodgroups = OrderedDict([
  ('em', ['em:None', 'em:TS', 'em:NBVS', 'em:BCTS', 'em:VS']),
  ('bbse', ['bbse-hard:None', 'bbse-soft:None',
            'bbse-soft:TS', 'bbse-soft:NBVS',
            'bbse-soft:BCTS', 'bbse-soft:VS'])])
samplesizes_in_table = ['7000', '8500', '10000']
adaptname_to_nicename = {'em': 'EM',
                         'bbse-soft': 'BBSE-soft',
                         'bbse-hard': 'BBSE-hard'}
calibname_to_nicename = {'None': 'None',
                           'TS': 'TS',
                           'NBVS': 'NBVS',
                           'BCTS': 'BCTS',
                           'VS': 'VS',
                           'best-nll':'Best NLL',
                           'best-jsdiv':'Best JS Div',
                           'best-ece':'Best ECE'}

dirichlet_alphas_in_table = ['0.1', '1.0']

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR100 Metric: JS Divergence, dirichlet shift",
    label="cifar100jsdivdirichletshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_acc',
    largerisbetter=True,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR100 Metric: $\\Delta$\\%Accuracy, dirichlet shift",
    label="cifar100deltaaccdirichletshift",
    applyunderline=False))




\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\alpha=0.1$} & \multicolumn{3}{| c}{$\alpha=1.0$}\\ 
    \cline{3-8}
    & & $n$=7000 & $n$=8500 & $n$=10000 & $n$=7000 & $n$=8500 & $n$=10000\\
    \hline
    \hline
    EM & None & inf; 4.0 & inf; 4.0 & inf; 4.0 & 0.244; 4.0 & 0.244; 4.0 & 0.243; 4.0\\
    EM & TS & 0.128; 1.58 & 0.129; 1.6 & 0.128; 1.56 & 0.122; 1.82 & 0.119; 1.82 & 0.12; 1.82\\
    EM & NBVS & 0.122; 1.8 & 0.122; 1.76 & 0.122; 1.82 & 0.121; 2.06 & 0.119; 2.12 & 0.119; 2.24\\
    EM & BCTS & 0.121; 1.48 & 0.121; 1.66 & 0.121; 1.64 & 0.117; 1.42 & 0.115; 1.52 & 0.115; 1.46\\
    EM & VS & \textbf{0.115; 1.14} & \textbf{0.114; 0.98} & \textbf{0.114; 0.98} & \textbf{0.113; 0.7} & \textbf{0.11; 0.54} & \textbf{0.11;

/usr/local/lib/python3.6/dist-packages/abstention/figure_making_utils.py:25: RuntimeWarning: invalid value encountered in subtract
  if (np.sum(np.abs(np.array(vals1)-np.array(vals2)))==0):
/usr/local/lib/python3.6/dist-packages/abstention/figure_making_utils.py:10: RuntimeWarning: invalid value encountered in subtract
  enumerate(sorted(vals1-vals2, key=lambda x: abs(x)))])


In [22]:
methodgroups = OrderedDict([
  ('em', ['em:NBVS', 'em:BCTS']),
  ('bbse', ['bbse-soft:NBVS',
            'bbse-soft:BCTS'])])

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR100 NVBS vs BCTS Metric: JS Divergence, dirichlet shift",
    label="cifar100nvbsbctsjsdivdirichletshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_acc',
    largerisbetter=True,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR100 NVBS vs BCTS Metric: $\\Delta$\\%Accuracy, dirichlet shift",
    label="cifar100nvbsbctsdeltaaccdirichletshift",
    applyunderline=False))



\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\alpha=0.1$} & \multicolumn{3}{| c}{$\alpha=1.0$}\\ 
    \cline{3-8}
    & & $n$=7000 & $n$=8500 & $n$=10000 & $n$=7000 & $n$=8500 & $n$=10000\\
    \hline
    \hline
    EM & NBVS & \textbf{0.122; 0.62} & \textbf{0.122; 0.6} & \textbf{0.122; 0.6} & 0.121; 0.74 & 0.119; 0.72 & 0.119; 0.8\\
    EM & BCTS & \textbf{0.121; 0.38} & \textbf{0.121; 0.4} & \textbf{0.121; 0.4} & \textbf{0.117; 0.26} & \textbf{0.115; 0.28} & \textbf{0.115; 0.2}\\
    \hline
    \hline
    BBSE-soft & NBVS & 0.161; 0.7 & 0.157; 0.68 & 0.158; 0.68 & 0.151; 0.94 & 0.146; 0.92 & 0.144; 0.96\\
    BBSE-soft & BCTS & \textbf{0.157; 0.3} & \textbf{0.154; 0.32} & \textbf{0.154; 0.32} & \textbf{0.147; 0.06} & \textb

In [18]:
methodgroups = OrderedDict([
  ('em-calib', ['em:best-nll', 'em:best-ece']),
  ('bbse-calib', ['bbse-soft:best-nll', 'bbse-soft:best-ece'])])

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_acc',
    largerisbetter=True,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR100 NLL vs ECE $\\Delta$\\%Accuracy, dirichlet shift",
    label="cifar100_nllvsece_deltaacc_dirichletshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR100 NLL vs ECE $\\Delta$\\%Accuracy, dirichlet shift",
    label="cifar100_nllvsece_deltaacc_dirichletshift",
    applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\alpha=0.1$} & \multicolumn{3}{| c}{$\alpha=1.0$}\\ 
    \cline{3-8}
    & & $n$=7000 & $n$=8500 & $n$=10000 & $n$=7000 & $n$=8500 & $n$=10000\\
    \hline
    \hline
    EM & Best NLL & \textbf{27.285; 0.3} & \textbf{27.288; 0.28} & \textbf{27.383; 0.0} & \textbf{21.763; 0.28} & \textbf{21.926; 0.16} & \textbf{22.074; 0.12}\\
    EM & Best ECE & 26.619; 0.7 & 26.497; 0.72 & \textbf{27.383; 1.0} & 21.493; 0.72 & 21.525; 0.84 & 21.616; 0.88\\
    \hline
    \hline
    BBSE-soft & Best NLL & \textbf{26.553; 0.32} & \textbf{26.805; 0.28} & \textbf{26.613; 0.0} & \textbf{21.328; 0.34} & \textbf{21.612; 0.24} & \textbf{21.717; 0.24}\\
    BBSE-soft & Best ECE & \textbf{26.084; 0.68} & 2